In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be s

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [3]:
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, random_split, DataLoader
from PIL import Image
import torchvision.models as models
from tqdm.notebook import tqdm
import torchvision.transforms as T
import torch.nn.functional as F
import torch.nn as nn
from torchvision.utils import make_grid

import os
import re
import requests

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [4]:
model_category = 'Women Tops & Tunics'

In [ ]:
train_df_Men_Tshirts = pd.read_csv('../../Preprocessor-FillNA/output/train_fillna_Women Tops Tunics_effnet_b5_4epochs.csv')
# train_df_Men_Tshirts = train_df_Men_Tshirts.head(100)

In [6]:
attr_columns = train_df_Men_Tshirts.filter(like='attr_').columns.to_list() # Adjust if more attributes exist
print(attr_columns)

['attr_1', 'attr_2', 'attr_3', 'attr_4', 'attr_5', 'attr_6', 'attr_7', 'attr_8', 'attr_9', 'attr_10']


In [7]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder for each attribute column
label_encoders = {}
for column in attr_columns:
    le = LabelEncoder()
    train_df_Men_Tshirts[column] = le.fit_transform(train_df_Men_Tshirts[column])
    label_encoders[column] = le  # Store the encoder for inverse transformation later if needed

# Check the updated DataFrame
# train_df_Men_Tshirts = train_df_Men_Tshirts.head()

In [8]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and validation sets
train_df1, val_df1 = train_test_split(train_df_Men_Tshirts, test_size=0.15, random_state=42)

In [9]:
train_df1

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10,image_path
12525,63900,Women Tops & Tunics,10,8,3,1,2,0,0,0,1,2,2,/kaggle/input/visual-taxonomy/train_images/063...
8406,59781,Women Tops & Tunics,10,5,3,1,2,0,0,0,1,2,3,/kaggle/input/visual-taxonomy/train_images/059...
17346,68721,Women Tops & Tunics,10,6,2,0,6,0,2,0,0,3,2,/kaggle/input/visual-taxonomy/train_images/068...
7588,58963,Women Tops & Tunics,10,8,2,0,1,0,2,4,2,3,2,/kaggle/input/visual-taxonomy/train_images/058...
17772,69147,Women Tops & Tunics,10,2,2,1,5,0,2,4,1,1,2,/kaggle/input/visual-taxonomy/train_images/069...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,62659,Women Tops & Tunics,10,2,3,1,6,0,2,4,1,1,2,/kaggle/input/visual-taxonomy/train_images/062...
11964,63339,Women Tops & Tunics,10,10,2,0,2,0,1,3,1,2,4,/kaggle/input/visual-taxonomy/train_images/063...
5390,56765,Women Tops & Tunics,10,1,3,1,6,0,2,4,1,0,2,/kaggle/input/visual-taxonomy/train_images/056...
860,52235,Women Tops & Tunics,10,2,1,1,0,0,0,1,3,2,5,/kaggle/input/visual-taxonomy/train_images/052...


In [10]:
val_df1

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10,image_path
1911,53286,Women Tops & Tunics,10,4,2,1,1,0,2,4,1,2,2,/kaggle/input/visual-taxonomy/train_images/053...
13144,64519,Women Tops & Tunics,10,8,3,1,1,0,1,1,3,0,2,/kaggle/input/visual-taxonomy/train_images/064...
2929,54304,Women Tops & Tunics,10,1,3,1,2,0,2,4,0,2,2,/kaggle/input/visual-taxonomy/train_images/054...
2968,54343,Women Tops & Tunics,10,2,3,1,1,0,2,4,1,2,0,/kaggle/input/visual-taxonomy/train_images/054...
10407,61782,Women Tops & Tunics,10,3,3,0,2,0,2,4,0,2,2,/kaggle/input/visual-taxonomy/train_images/061...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18011,69386,Women Tops & Tunics,10,8,1,1,0,1,2,4,2,0,1,/kaggle/input/visual-taxonomy/train_images/069...
6844,58219,Women Tops & Tunics,10,8,2,1,3,0,2,4,1,2,5,/kaggle/input/visual-taxonomy/train_images/058...
13951,65326,Women Tops & Tunics,10,10,0,0,2,0,1,5,1,2,1,/kaggle/input/visual-taxonomy/train_images/065...
9326,60701,Women Tops & Tunics,10,11,3,0,2,0,1,5,2,3,1,/kaggle/input/visual-taxonomy/train_images/060...


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from PIL import Image
import pandas as pd
import numpy as np

# Define custom dataset
class CustomDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, is_test=False):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test  # Flag to indicate if it's test set without labels

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.img_dir + self.dataframe.iloc[idx]['image_path']
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        if self.is_test:  # For test set, just return the image without labels
            return image
        
        # For train/validation set, return image and labels
        labels = self.dataframe.iloc[idx][attr_columns].values
        labels = labels.astype(np.int64)  # Ensure labels are integers
        labels = torch.tensor(labels, dtype=torch.long)
        
        return image, labels
    
    
# Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Apply Color Jitter
    transforms.RandomHorizontalFlip(),  # Apply Horizontal Flip with 50% probability
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
    
train_dataset = CustomDataset(dataframe=train_df1, img_dir='', transform=transform)
val_dataset = CustomDataset(dataframe=val_df1, img_dir='', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [12]:
import ssl

ssl._create_default_https_context = ssl._create_stdlib_context

In [13]:
from torchvision import models
available_models = dir(models)

# Filter out private members and display model names
available_models = [model for model in available_models if not model.startswith('_')]
print("Available models in torchvision.models:")
print(available_models)

Available models in torchvision.models:
['AlexNet', 'AlexNet_Weights', 'ConvNeXt', 'ConvNeXt_Base_Weights', 'ConvNeXt_Large_Weights', 'ConvNeXt_Small_Weights', 'ConvNeXt_Tiny_Weights', 'DenseNet', 'DenseNet121_Weights', 'DenseNet161_Weights', 'DenseNet169_Weights', 'DenseNet201_Weights', 'EfficientNet', 'EfficientNet_B0_Weights', 'EfficientNet_B1_Weights', 'EfficientNet_B2_Weights', 'EfficientNet_B3_Weights', 'EfficientNet_B4_Weights', 'EfficientNet_B5_Weights', 'EfficientNet_B6_Weights', 'EfficientNet_B7_Weights', 'EfficientNet_V2_L_Weights', 'EfficientNet_V2_M_Weights', 'EfficientNet_V2_S_Weights', 'GoogLeNet', 'GoogLeNetOutputs', 'GoogLeNet_Weights', 'Inception3', 'InceptionOutputs', 'Inception_V3_Weights', 'MNASNet', 'MNASNet0_5_Weights', 'MNASNet0_75_Weights', 'MNASNet1_0_Weights', 'MNASNet1_3_Weights', 'MaxVit', 'MaxVit_T_Weights', 'MobileNetV2', 'MobileNetV3', 'MobileNet_V2_Weights', 'MobileNet_V3_Large_Weights', 'MobileNet_V3_Small_Weights', 'RegNet', 'RegNet_X_16GF_Weights', '

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

class MultiOutputModelSwin(nn.Module):
    def __init__(self, num_classes):
        super(MultiOutputModelSwin, self).__init__()
        
        # Load a pretrained Swin V2 model
        self.base_model = models.swin_v2_b(weights="IMAGENET1K_V1")
        
        # Remove the original classification head
        in_features = self.base_model.head.in_features  # Get the input features for the head
        self.base_model.head = nn.Identity()  # Replace with identity to extract embeddings
        
        # Define output layers for each attribute dynamically
        self.output_layers = nn.ModuleDict()
        for attr, n_classes in num_classes.items():
            self.output_layers[attr] = nn.Linear(in_features, n_classes)  # Use `in_features` for each linear layer

    def forward(self, x):
        # Pass input through the Swin model to get embeddings
        x = self.base_model(x)
        
        # Compute output for each attribute
        outputs = {attr: layer(x) for attr, layer in self.output_layers.items()}
        
        return list(outputs.values())

# Example usage
# Define number of classes for each attribute
num_classes = {attr: train_df_Men_Tshirts[attr].nunique() for attr in attr_columns}

# Initialize the model
model = MultiOutputModelSwin(num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


Downloading: "https://download.pytorch.org/models/swin_v2_b-781e5279.pth" to /root/.cache/torch/hub/checkpoints/swin_v2_b-781e5279.pth
100%|██████████| 336M/336M [00:01<00:00, 212MB/s]


In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Training loop
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, save_path="Kurtis_swin_v2.pth"):
    scheduler = optim.lr_scheduler.StepLR(optimizer = optimizer , step_size=5, gamma=0.5)
    best_val_loss = float('inf')
    for epoch in tqdm(range(num_epochs)):
        model.train()
        train_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)  # Move tensors to the correct device
            optimizer.zero_grad()
            outputs = model(images)
            loss = sum([criterion(output, label) for output, label in zip(outputs, labels.T)])
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        scheduler.step()
        
        print(f'Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader)}')

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)  # Move tensors to the correct device
                outputs = model(images)
                loss = sum([criterion(output, label) for output, label in zip(outputs, labels.T)])
                val_loss += loss.item()
        
        avg_val_loss = val_loss / len(val_loader)
        print(f'Epoch {epoch+1}, Val Loss: {val_loss/len(val_loader)}')
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), save_path)
            print(f"Model saved with Val Loss: {best_val_loss:.4f} at Epoch {epoch+1}")

# Run training
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=6)

model.load_state_dict(torch.load('Kurtis_swin_v2.pth'))

  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1, Train Loss: 5.61588830003644
Epoch 1, Val Loss: 3.757932694753011
Model saved with Val Loss: 3.7579 at Epoch 1
Epoch 2, Train Loss: 3.5287025073967357
Epoch 2, Val Loss: 3.3184305005603365
Model saved with Val Loss: 3.3184 at Epoch 2
Epoch 3, Train Loss: 2.9758388202969392
Epoch 3, Val Loss: 3.101348070303599
Model saved with Val Loss: 3.1013 at Epoch 3
Epoch 4, Train Loss: 2.6105849072484686
Epoch 4, Val Loss: 3.0373619430594974
Model saved with Val Loss: 3.0374 at Epoch 4
Epoch 5, Train Loss: 2.3497180414671943
Epoch 5, Val Loss: 3.0149019718170167
Model saved with Val Loss: 3.0149 at Epoch 5
Epoch 6, Train Loss: 1.8650764283567371
Epoch 6, Val Loss: 2.9211394482188755
Model saved with Val Loss: 2.9211 at Epoch 6


<All keys matched successfully>

In [16]:
test_df = pd.read_csv('/kaggle/input/visual-taxonomy/test.csv')
# test_df = test_df.sample(100)

In [17]:
test_df_Men_Tshirts = test_df[test_df['Category'] == model_category]
test_df_Men_Tshirts['len'] = 8

def format_image_path_test(row):
    return f"/kaggle/input/visual-taxonomy/test_images/{str(row['id']).zfill(6)}.jpg"

test_df_Men_Tshirts['image_path'] = test_df_Men_Tshirts.apply(format_image_path_test, axis=1)
test_df_Men_Tshirts

,id,Category,len,image_path
23280,23564,Women Tops & Tunics,8,/kaggle/input/visual-taxonomy/test_images/0235...
23281,23565,Women Tops & Tunics,8,/kaggle/input/visual-taxonomy/test_images/0235...
23282,23566,Women Tops & Tunics,8,/kaggle/input/visual-taxonomy/test_images/0235...
23283,23567,Women Tops & Tunics,8,/kaggle/input/visual-taxonomy/test_images/0235...
23284,23568,Women Tops & Tunics,8,/kaggle/input/visual-taxonomy/test_images/0235...
...,...,...,...,...
30200,30484,Women Tops & Tunics,8,/kaggle/input/visual-taxonomy/test_images/0304...
30201,30485,Women Tops & Tunics,8,/kaggle/input/visual-taxonomy/test_images/0304...
30202,30486,Women Tops & Tunics,8,/kaggle/input/visual-taxonomy/test_images/0304...
30203,30487,Women Tops & Tunics,8,/kaggle/input/visual-taxonomy/test_images/0304...


In [18]:
# Test dataset without labels
test_dataset = CustomDataset(dataframe=test_df_Men_Tshirts, img_dir='', transform=transform, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [19]:
# Assuming model and test_loader are defined, and num_classes is available
predictions = []

with torch.no_grad():  # No need to compute gradients during inference
    for images in test_loader:  # Only images, no labels for the test set
        images = images.cuda() if torch.cuda.is_available() else images
        
        # Forward pass through the model
        outputs = model(images)  # Outputs will be a tuple based on the number of attributes
        
        # Initialize a list to hold predictions for the batch
        batch_preds = []
        
        # Loop through the outputs to get predictions for each attribute
        for out in outputs:
            _, pred = torch.max(out, 1)  # Get predicted classes for the current output
            batch_preds.append(pred.cpu().numpy())  # Move predictions to CPU and convert to numpy
            
        # Stack the predictions along a new dimension and append to the predictions list
        predictions.append(np.stack(batch_preds, axis=1))  # Shape: (batch_size, num_attributes)

# Combine all predictions into a single array
predictions = np.concatenate(predictions, axis=0)

# Display predictions
print(predictions)  # This will be an array with shape (num_samples, num_attributes)


[[ 6  2  0 ...  2  3  2]
 [11  0  0 ...  1  2  1]
 [ 0  2  1 ...  0  2  0]
 ...
 [ 2  3  1 ...  1  0  2]
 [ 5  3  1 ...  0  2  3]
 [ 8  2  0 ...  2  0  2]]


In [20]:
# Assuming predictions is a numpy array of shape (N, 5)
# Add new columns attr_1 to attr_10 to test_df
for i in range(1, 11):
    test_df_Men_Tshirts[f'attr_{i}'] = np.nan

# Assign predictions to attr_1 to attr_5
test_df_Men_Tshirts[attr_columns] = predictions

# Optionally save the updated test_df to CSV
# test_df.to_csv('test_predictions_with_attrs.csv', index=False)

# Display the first few rows of the updated DataFrame
test_df_Men_Tshirts.head()

,id,Category,len,image_path,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
23280,23564,Women Tops & Tunics,8,/kaggle/input/visual-taxonomy/test_images/0235...,6,2,0,1,0,0,0,2,3,2
23281,23565,Women Tops & Tunics,8,/kaggle/input/visual-taxonomy/test_images/0235...,11,0,0,2,0,1,2,1,2,1
23282,23566,Women Tops & Tunics,8,/kaggle/input/visual-taxonomy/test_images/0235...,0,2,1,2,0,2,4,0,2,0
23283,23567,Women Tops & Tunics,8,/kaggle/input/visual-taxonomy/test_images/0235...,8,3,1,6,0,2,4,1,0,2
23284,23568,Women Tops & Tunics,8,/kaggle/input/visual-taxonomy/test_images/0235...,0,2,1,2,0,0,4,1,2,2


In [21]:
# Inverse transform predictions for each attribute using the stored label encoders
for attr in attr_columns:
    # Inverse transform using the corresponding label encoder
    test_df_Men_Tshirts[attr] = label_encoders[attr].inverse_transform(test_df_Men_Tshirts[attr].astype(int))

# Check the updated DataFrame with original attribute names instead of encoded numbers
test_df_Men_Tshirts

,id,Category,len,image_path,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
23280,23564,Women Tops & Tunics,8,/kaggle/input/visual-taxonomy/test_images/0235...,navy blue,fitted,crop,high,casual,default,default,sleeveless,sleeveless,knitted
23281,23565,Women Tops & Tunics,8,/kaggle/input/visual-taxonomy/test_images/0235...,yellow,boxy,crop,round neck,casual,printed,graphic,short sleeves,regular sleeves,default
23282,23566,Women Tops & Tunics,8,/kaggle/input/visual-taxonomy/test_images/0235...,black,fitted,regular,round neck,casual,solid,solid,long sleeves,regular sleeves,applique
23283,23567,Women Tops & Tunics,8,/kaggle/input/visual-taxonomy/test_images/0235...,pink,regular,regular,v-neck,casual,solid,solid,short sleeves,default,knitted
23284,23568,Women Tops & Tunics,8,/kaggle/input/visual-taxonomy/test_images/0235...,black,fitted,regular,round neck,casual,default,solid,short sleeves,regular sleeves,knitted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30200,30484,Women Tops & Tunics,8,/kaggle/input/visual-taxonomy/test_images/0304...,green,boxy,crop,round neck,casual,printed,typography,short sleeves,regular sleeves,tie-ups
30201,30485,Women Tops & Tunics,8,/kaggle/input/visual-taxonomy/test_images/0304...,default,regular,regular,round neck,casual,solid,solid,short sleeves,puff sleeves,knitted
30202,30486,Women Tops & Tunics,8,/kaggle/input/visual-taxonomy/test_images/0304...,default,regular,regular,v-neck,casual,solid,solid,short sleeves,default,knitted
30203,30487,Women Tops & Tunics,8,/kaggle/input/visual-taxonomy/test_images/0304...,multicolor,regular,regular,round neck,casual,default,default,long sleeves,regular sleeves,ruffles


In [22]:
# Get value counts for each specified column
columns_of_interest = attr_columns

for column in columns_of_interest:
    print(f"Value counts for {column}:\n")
    print(test_df_Men_Tshirts[column].value_counts(dropna=False))  # Including NaN values
    print("\n" + "-"*50 + "\n")

Value counts for attr_1:

attr_1
black         915
white         807
pink          793
default       749
blue          647
red           639
yellow        591
maroon        534
green         488
navy blue     284
peach         255
multicolor    223
Name: count, dtype: int64

--------------------------------------------------

Value counts for attr_2:

attr_2
fitted     2961
regular    2468
boxy        824
default     672
Name: count, dtype: int64

--------------------------------------------------

Value counts for attr_3:

attr_3
regular    4433
crop       2492
Name: count, dtype: int64

--------------------------------------------------

Value counts for attr_4:

attr_4
round neck         2763
square neck         890
high                775
default             699
sweetheart neck     697
v-neck              687
stylised            414
Name: count, dtype: int64

--------------------------------------------------

Value counts for attr_5:

attr_5
casual    6753
party      172
Name: cou

In [23]:
# import matplotlib.pyplot as plt
# import pandas as pd
# import matplotlib.image as mpimg

# # Sample 5 random images from the DataFrame
# sample_df = test_df_Men_Tshirts.sample(5)

# # Create subplots to display the images with their labels
# fig, axes = plt.subplots(1, 5, figsize=(20, 10))  # 5 images, side by side

# # Loop through each image and label
# for i, (idx, row) in enumerate(sample_df.iterrows()):
#     # Read the image
#     img = mpimg.imread(row['image_path'])
    
#     # Display the image in the subplot
#     axes[i].imshow(img)
#     axes[i].axis('off')  # Turn off the axis for a cleaner look
    
# #     Set the title with the labels
#     label = ""
#     for attr, renamed_key in rename_dict.items():
#         label += f"{renamed_key.capitalize()}: {row[attr]}\n"

#     # Remove the last newline character
#     label = label.strip()
#     axes[i].set_title(label, fontsize=10)

# # Show the plot
# plt.tight_layout()
# plt.show()


In [24]:
sub_df = pd.read_csv('/kaggle/input/visual-taxonomy/sample_submission.csv')
sub_df

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
0,47001,Women Tshirts,8,maroon,regular,crop,printed,typography,short sleeves,regular sleeves,default,nu,nu
1,16431,Sarees,10,same as saree\n,temple border,small border,cream,party,jacquard,woven design,zari woven,ethnic motif,no
2,55700,Women Tops & Tunics,10,white,fitted,regular,round neck,casual,solid,solid,short sleeves,regular sleeves,knitted
3,15698,Sarees,10,same as saree\n,zari,small border,white,party,jacquard,woven design\n,zari woven,floral,no
4,30330,Kurtis,9,yellow,a-line,knee length\n,daily,net,default,solid,three-quarter sleeves,regular,nu


In [25]:
# sub_df_Men_Tshirts.info()

In [26]:
sub_df_Men_Tshirts = test_df_Men_Tshirts.drop('image_path', axis = 1)
# sub_df_Men_Tshirts.replace("unknown", np.nan, inplace=True)
sub_df_Men_Tshirts.to_csv(f"sub_df_{model_category}_vit_l16_fillna_4epochs_alyal.csv", index = False)
sub_df_Men_Tshirts

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
23280,23564,Women Tops & Tunics,8,navy blue,fitted,crop,high,casual,default,default,sleeveless,sleeveless,knitted
23281,23565,Women Tops & Tunics,8,yellow,boxy,crop,round neck,casual,printed,graphic,short sleeves,regular sleeves,default
23282,23566,Women Tops & Tunics,8,black,fitted,regular,round neck,casual,solid,solid,long sleeves,regular sleeves,applique
23283,23567,Women Tops & Tunics,8,pink,regular,regular,v-neck,casual,solid,solid,short sleeves,default,knitted
23284,23568,Women Tops & Tunics,8,black,fitted,regular,round neck,casual,default,solid,short sleeves,regular sleeves,knitted
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30200,30484,Women Tops & Tunics,8,green,boxy,crop,round neck,casual,printed,typography,short sleeves,regular sleeves,tie-ups
30201,30485,Women Tops & Tunics,8,default,regular,regular,round neck,casual,solid,solid,short sleeves,puff sleeves,knitted
30202,30486,Women Tops & Tunics,8,default,regular,regular,v-neck,casual,solid,solid,short sleeves,default,knitted
30203,30487,Women Tops & Tunics,8,multicolor,regular,regular,round neck,casual,default,default,long sleeves,regular sleeves,ruffles
